 
 Tweet Sentiment NER approach
 
 Inspired from -

 https://www.depends-on-the-definition.com/named-entity-recognition-with-bert/

https://www.kaggle.com/gskdhiman/bert-baseline-starter-kernel-ner-approach


In [ ]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
link='https://drive.google.com/open?id=1YK66P0Oi-oF-untCgXFtIO3062DGlQHY'
fluff, id = link.split('=')
print (id)

1YK66P0Oi-oF-untCgXFtIO3062DGlQHY


In [ ]:
# Install required packages
! pip install transformers==2.8.0
import os
import gc
import numpy as np 
import pandas as pd
import torch
import tensorflow as tf
import tensorflow.keras.backend as K
from transformers import BertTokenizer
from transformers import BertTokenizer,BertConfig,TFBertModel
from tqdm import tqdm
#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)


In [ ]:
# Load vocab file
! wget https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt

--2020-06-16 16:10:39--  https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.8.126
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.8.126|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 231508 (226K) [text/plain]
Saving to: ‘bert-base-uncased-vocab.txt’

bert-base-uncased-v 100%[===================>] 226.08K   813KB/s    in 0.3s    

2020-06-16 16:10:40 (813 KB/s) - ‘bert-base-uncased-vocab.txt’ saved [231508/231508]



In [ ]:
import tokenizers
tokenizer=tokenizers.BertWordPieceTokenizer("bert-base-uncased-vocab.txt",lowercase=True)
#tokenizer=tokenizer.add_special_tokens([ "[SEP]", "[UNK]", "[T1]", "[T2]" ])

In [ ]:
# Get the GPU device name.set_device()
device_name = tf.test.gpu_device_name()
# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

# If there's a GPU available...
if torch.cuda.is_available():    
    # Tell PyTorch to use the GPU.    
    #device = torch.cuda.set_device(0)
    device = torch.device('cuda:0')

    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

Found GPU at: /device:GPU:0
There are 1 GPU(s) available.
We will use the GPU: Tesla K80


In [ ]:
### data preprocessing
import pandas as pd
import numpy as np
downloaded = drive.CreateFile({'id':id}) 

downloaded.GetContentFile('train.csv')  
df = pd.read_csv('train.csv')
df=df.drop([i for i in range(len(df['text'])) if  df['text'][i] in ['nan', np.nan]])


In [ ]:
def process_data(tweet, selected_text, tokenizer):
  """ This function will create target list ,selected text as 1 and otherwise 0
  """
    len_st = len(selected_text)
    idx0 = None
    idx1 = None

    for ind in (i for i, e in enumerate(tweet) if e == selected_text[0]):
        if tweet[ind: ind+len_st] == selected_text:
            idx0 = ind
            idx1 = ind + len_st
            break

    char_targets = [0] * len(tweet)
    if idx0 != None and idx1 != None:
        for ct in range(idx0, idx1):
            char_targets[ct] = 1
            
    tok_tweet = tokenizer.encode(tweet)
    input_ids_orig = tok_tweet.ids
    tweet_offsets = tok_tweet.offsets

    target_idx = []
    for j, (offset1, offset2) in enumerate(tweet_offsets):
        if sum(char_targets[offset1: offset2]) > 0:
            target_idx.append(j)
            
    targets = [0] * len(input_ids_orig)
    for idx in target_idx:
        targets[idx] = 1
    return targets

In [ ]:
df['targets'] = df.apply(lambda row: process_data(str(row['text']), 
                                                                    str(row['selected_text']),
                                                                    tokenizer),
                                                                    axis=1)
max_len=128
df['targets'] = df['targets'].apply(lambda x :x + [0] * (max_len-len(x)))

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
#from transformers import BertTokenizer, BertConfig
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer
tokenizer_brt = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [ ]:
%%time
# Create input token ids, attention mask and target tags and keep all of them in dictionary
text_tag_list=[]
for id in df['textID']:
  tweet=df[df['textID']==id]['text'].values[0]
  tag=df['targets'][df['textID']==id].values[0]
  tweet_token=tokenizer_brt.encode_plus(tweet,max_length=128,pad_to_max_length=True,return_attention_mask = True)
  text_tag_list.append({'id':id,'tweet':tweet,'tweet_token':tweet_token['input_ids'],'attention_mask':tweet_token['attention_mask'],'tag':tag})

CPU times: user 2min 2s, sys: 123 ms, total: 2min 2s
Wall time: 2min 2s


In [ ]:
# Text train split and converting to tensor

train_text_tag,val_text_tag=train_test_split(text_tag_list,random_state=2018,test_size=.1)
tr_inputs=torch.tensor([train_text_tag[i]['tweet_token'] for i in range(len(train_text_tag))])
val_inputs=torch.tensor([val_text_tag[i]['tweet_token'] for i in range(len(val_text_tag))])
tr_tags=torch.tensor([train_text_tag[i]['tag'] for i in range(len(train_text_tag))])
val_tags=torch.tensor([val_text_tag[i]['tag'] for i in range(len(val_text_tag))])
tr_masks=torch.tensor([train_text_tag[i]['attention_mask'] for i in range(len(train_text_tag))])
val_masks=torch.tensor([val_text_tag[i]['attention_mask'] for i in range(len(val_text_tag))])

In [ ]:
# Pytorch data loader to load train and validation sheet
bs = 32
train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=bs)

valid_data = TensorDataset(val_inputs, val_masks, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=bs)


Build Model


In [2]:
from transformers import BertForTokenClassification, AdamW

In [ ]:
model = BertForTokenClassification.from_pretrained(
    "bert-base-cased",
    num_labels=2,
    output_attentions = False,
    output_hidden_states = False
)
##model.resize_token_embeddings(len(tokenizer_brt))
#model.cuda()

In [ ]:
model.resize_token_embeddings(len(tokenizer))
model.cuda()

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)


In [ ]:
FULL_FINETUNING = True
if FULL_FINETUNING:
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    param_optimizer = list(model.classifier.named_parameters())
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]

optimizer = AdamW(
    optimizer_grouped_parameters,
    lr=3e-5,
    eps=1e-8
)


In [ ]:
from transformers import get_linear_schedule_with_warmup

epochs = 1
max_grad_norm = 1.0

# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)


Train

In [ ]:
%%time
## Store the average loss after each epoch so we can plot them.
loss_values, validation_loss_values = [], []

for i in range(0,epochs):
    # ========================================
    #               Training
    # ========================================
    model.train()
    total_loss = 0
    for step, batch in enumerate(train_dataloader):
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        model.zero_grad()
        outputs = model(b_input_ids, token_type_ids=None,
                        attention_mask=b_input_mask, labels=b_labels)
        loss = outputs[0]
        print(loss,step)
        total_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        optimizer.step()
        scheduler.step()
    avg_train_loss = total_loss / len(train_dataloader)
    print("Average train loss: {}".format(avg_train_loss))
    loss_values.append(avg_train_loss)


    # ========================================
    #               Validation
    # ========================================
    model.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    predictions , true_labels = [], []
    for batch in valid_dataloader:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        with torch.no_grad():
            # Forward pass, calculate logit predictions.
            # This will return the logits rather than the loss because we have not provided labels.
            outputs = model(b_input_ids, token_type_ids=None,
                            attention_mask=b_input_mask, labels=b_labels)
        # Move logits and labels to CPU
        logits = outputs[1].detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences.
        eval_loss += outputs[0].mean().item()
        print("eval loss  :"+str(eval_loss))
        predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
        true_labels.extend(label_ids)

    eval_loss = eval_loss / len(valid_dataloader)
    validation_loss_values.append(eval_loss)
    #print("Validation loss: {}".format(eval_loss))
    #pred_tags = [tag_values[p_i] for p, l in zip(predictions, true_labels)
    #                             for p_i, l_i in zip(p, l) if tag_values[l_i] != "PAD"]
    #valid_tags = [tag_values[l_i] for l in true_labels
    #                              for l_i in l if tag_values[l_i] != "PAD"]
    #print("Validation Accuracy: {}".format(accuracy_score(pred_tags, valid_tags)))
    #print("Validation F1-Score: {}".format(f1_score(pred_tags, valid_tags)))
    print()


tensor(0.7593, device='cuda:0', grad_fn=<NllLossBackward>) 0
tensor(0.6898, device='cuda:0', grad_fn=<NllLossBackward>) 1
tensor(0.7739, device='cuda:0', grad_fn=<NllLossBackward>) 2
tensor(0.6959, device='cuda:0', grad_fn=<NllLossBackward>) 3
tensor(0.7218, device='cuda:0', grad_fn=<NllLossBackward>) 4
tensor(0.6577, device='cuda:0', grad_fn=<NllLossBackward>) 5
tensor(0.6510, device='cuda:0', grad_fn=<NllLossBackward>) 6
tensor(0.7000, device='cuda:0', grad_fn=<NllLossBackward>) 7
tensor(0.8127, device='cuda:0', grad_fn=<NllLossBackward>) 8
tensor(0.6430, device='cuda:0', grad_fn=<NllLossBackward>) 9
tensor(0.6437, device='cuda:0', grad_fn=<NllLossBackward>) 10
tensor(0.6660, device='cuda:0', grad_fn=<NllLossBackward>) 11
tensor(0.6599, device='cuda:0', grad_fn=<NllLossBackward>) 12
tensor(0.6222, device='cuda:0', grad_fn=<NllLossBackward>) 13
tensor(0.6980, device='cuda:0', grad_fn=<NllLossBackward>) 14
tensor(0.6022, device='cuda:0', grad_fn=<NllLossBackward>) 15
tensor(0.6323, dev

In [ ]:
print(avg_train_loss)
print(eval_loss)

0.6092203807707437
0.584078751331152


In [ ]:
acc=[]
for j in range(len(valid_dataloader)):
 acc.append(sum([1 for i in range(0,128) if predictions[j][i]==true_labels[j][i]])/128)
sum(acc)/len(valid_dataloader)

0.42014898255813954

In [ ]:
def decode_tweet(original_tweet,idx_start,idx_end,offsets):
    filtered_output  = ""
    for ix in range(idx_start, idx_end + 1):
        filtered_output += original_tweet[offsets[ix][0]: offsets[ix][1]]
        if (ix+1) < len(offsets) and offsets[ix][1] < offsets[ix+1][0]:
            filtered_output += " "
    return filtered_output

Test

In [ ]:
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
id1='1hlqiVCKMApFC5WEexahUouBqFh5mi7b0'
print(id1)
downloaded = drive.CreateFile({'id':id1}) 
downloaded.GetContentFile('test.csv')  
test = pd.read_csv('test.csv')
test=test.drop([i for i in range(len(test['text'])) if  test['text'][i] in ['nan', np.nan]])
test.shape

1hlqiVCKMApFC5WEexahUouBqFh5mi7b0


(3534, 3)

In [ ]:
subm_list=[]
for test_ids in test['textID']:
  if test['sentiment'][test['textID']==test_ids].values[0]=='neutral':
    selected_text=test['text'][test['textID']==test_ids].values[0]
  else:
    test_token=tokenizer_brt.encode_plus(test['text'][test['textID']==test_ids].values[0],max_length=128,pad_to_max_length=True)
    test_input_ids = torch.tensor([test_token['input_ids']]).cuda()
    with torch.no_grad():
      output = model(test_input_ids)
    label_indices = np.argmax(output[0].to('cpu').numpy(), axis=2)
    indexes=[i for i in range(len(label_indices[0])) if label_indices[0][i]==1]
    encoded_text=tokenizer.encode(test['text'][test['textID']==test_ids].values[0])
    if len(indexes)>0:
      start = indexes[0]
      end =  indexes[-1]
    else:  
      start = 0
      end = len(encoded_text.ids) - 1
    if end >= len(encoded_text.ids): 
      end = len(encoded_text.ids) - 1
    #print(test_ids,start,end,len(encoded_test_token.offsets))
    selected_text=decode_tweet(test['text'][test['textID']==test_ids].values[0],start,end,encoded_text.offsets)
  subm_list.append({'textID':test_ids,'text':test['text'][test['textID']==test_ids].values[0],\
                     'sentiment':test['sentiment'][test['textID']==test_ids].values[0],'selected_text':selected_text})

Ready for Submission

In [ ]:
pd.DataFrame(subm_list)

,textID,text,sentiment,selected_text
0,f87dea47db,Last session of the day http://twitpic.com/67ezh,neutral,Last session of the day http://twitpic.com/67ezh
1,96d74cb729,Shanghai is also really exciting (precisely -...,positive,Good
2,eee518ae67,"Recession hit Veronique Branquinho, she has to...",negative,"Recession hit Veronique Branquinho, she has to..."
3,01082688c6,happy bday!,positive,happy
4,33987a8ee5,http://twitpic.com/4w75p - I like it!!,positive,http://twitpic.com/4w75p - I like it!!
...,...,...,...,...
3529,e5f0e6ef4b,"its at 3 am, im very tired but i can`t sleep ...",negative,"its at 3 am, im very tired but i can`t sleep b..."
3530,416863ce47,All alone in this old house again. Thanks for...,positive,Thanks
3531,6332da480c,I know what you mean. My little dog is sinkin...,negative,I know what you mean. My little dog is sinkin...
3532,df1baec676,_sutra what is your next youtube video gonna b...,positive,love
